# 📚 Kapitola 13: Práce s API a HTTP requests

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | INTERMEDIATE</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **Requests library - installation**
- **HTTP methods: GET, POST, PUT, DELETE, PATCH**
- **Headers a authentication**
- **Query parameters a request body**
- **JSON API consumption**
- **Error handling a status codes**
- **Rate limiting a retry logic**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 8

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 13

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 13: Práce s API a HTTP requests")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Práce s API a HTTP requests

## 1. Úvod a motivace

Práce s API (Application Programming Interface) je klíčovou součástí moderního vývoje softwaru, a proto je důležité ji zvládnout na úrovni intermédiatu. V praxi se API používají skoro všude – od mobilních aplikací přes webové služby až po systémy pro správu dat, sociální sítě, platobní brány a mnoho dalších. Vývojáři neustále komunikují s externími službami prostřednictvím HTTP requestů, které umožňují získávat, odesílat, upravovat a mazat data.

Ve výuce se tato kapitola připravuje na praktické aplikace, jako je tvorba webových aplikací, automatizace úkolů, analýza dat nebo propojení různých systémů. Student po této kapitole získá schopnost vytvářet a ovládat HTTP požadavky, pracovat s JSON daty, řešit chyby a zabezpečit komunikaci mezi aplikacemi – všechno to je zásadní dovednost pro každého vývojáře.

## 2. Hlavní koncepty

### Requests library - instalace

Pro práci s HTTP požadavky v Pythonu se používá knihovna `requests`. Je to jedna z nejrozšířenějších a nejjednodušších knihoven pro komunikaci přes HTTP. Instalace probíhá jednoduše přes pip:

```bash
pip install requests
```

Po instalaci je možné začít používat modul v kódu:

```python
import requests

response = requests.get('https://api.github.com')
print(response.status_code)
```

Knihovna `requests` umožňuje snadno odesílat HTTP požadavky a zpracovávat odpovědi. Je to nástroj, který vývojářům ušetří mnoho času, protože se stará o složitosti připojení, serializaci dat a řešení chyb za nás.

---

### HTTP metody: GET, POST, PUT, DELETE, PATCH

HTTP požadavky jsou založeny na metodách, které určují, co se má s prostředkem provést. Základní metody:

- **GET** – používá se pro získání dat (např. seznam uživatelů).
- **POST** – slouží k vytváření nových záznamů (např. registrace uživatele).
- **PUT** – aktualizuje celý zdroj (např. změna údajů uživatele).
- **DELETE** – odstraní zdroj (např. smazání uživatele).
- **PATCH** – částečná aktualizace zdroje (např. změna jen jednoho pole).

```python
import requests

# GET požadavek
response = requests.get('https://api.github.com/users/octocat')
print(response.json())

# POST požadavek
data = {'title': 'Nový příspěvek', 'body': 'Obsah příspěvku'}
response = requests.post('https://jsonplaceholder.typicode.com/posts', json=data)
print(response.status_code)
```

Každá metoda má své specifické použití a význam v rámci REST API.

---

### Headers a autentizace

HTTP hlavičky (`headers`) jsou důležité pro předání informací mezi klientem a serverem. Například:

```python
headers = {
    'Authorization': 'Bearer token123',
    'Content-Type': 'application/json'
}
response = requests.get('https://api.example.com/data', headers=headers)
```

Použití hlaviček je nezbytné pro autentizaci (např. pomocí Bearer tokenu), nastavení formátu dat, řízení cache a dalších funkcí.

---

### Query parametry a request body

Query parametry se přidávají ke konci URL adresy (např. `?page=1&limit=10`) a používají se pro filtrování nebo řazení dat:

```python
params = {'page': 1, 'limit': 10}
response = requests.get('https://api.example.com/users', params=params)
```

Request body se používá při metodách POST, PUT a PATCH, kde se data odesílají v těle požadavku:

```python
data = {'name': 'Jan Novák', 'email': 'jan@example.com'}
response = requests.post('https://api.example.com/users', json=data)
```

---

### JSON API consumption

Většina moderních API používá formát JSON pro přenos dat. Knihovna `requests` obsahuje metodu `.json()`, která automaticky převede odpověď na Python slovník:

```python
response = requests.get('https://api.github.com/users/octocat')
user_data = response.json()
print(user_data['name'])
```

Výsledek `response.json()` je Python objekt (např. slovník nebo seznam), který lze snadno zpracovávat.

---

### Error handling a status codes

HTTP odpovědi mají kódy statusu, které indikují úspěch nebo chybu:

- `200` – OK
- `404` – nenalezeno
- `500` – interní chyba serveru

Zpracování chyb se provádí např. takto:

```python
try:
    response = requests.get('https://api.example.com/data')
    response.raise_for_status()  # vyvolá výjimku při chybném kódu
except requests.exceptions.HTTPError as e:
    print(f"HTTP chyba: {e}")
except requests.exceptions.RequestException as e:
    print(f"Chyba při požadavku: {e}")
```

---

### Rate limiting a retry logika

Mnoho API má omezení počtu požadavků za jednotku času. Pokud se překročí limit, server odpoví např. kódem `429`. V takovém případě je vhodné implementovat retry logiku:

```python
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = requests.Session()
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

response = session.get('https://api.example.com/data')
```

Tento kód zajišťuje opakované pokusy o požadavek v případě chyb.

## 3. Důležité detaily

### Časté chyby a jak se jim vyhnout

- **Nepředávání hlaviček** – například chybějící `Authorization` token.
- **Nesprávné použití metody** – např. použití GET pro úpravu dat.
- **Nezachycení chyb** – výjimky při požadavku nemusí být správně zachyceny.
- **Nepoužívání session** – pro opakované požadavky je lepší použít `Session` objekt.

### Best practices

- Vždy používejte `requests.Session()` pro více požadavků.
- Zpracovávejte chyby přes `try-except`.
- Nastavujte timeout, aby se zabránilo zaseknutí aplikace:
  ```python
  response = requests.get('https://api.example.com', timeout=5)
  ```
- Užívejte `json` parametr při odesílání dat.

### Performance tipy

- Využívejte `requests.Session()` pro optimalizaci připojení.
- Vyhněte se opakovanému vytváření požadavků – používejte cache a batch operace.
- Používejte `headers` pro řízení cache, komprese a zabezpečení.

## 4. Propojení s předchozími kapitolami

Tato kapitola navazuje na předchozí znalosti jako je práce se slovníky, seznamy, výjimkami a funkcemi. Naučí studenty, jak tyto koncepty použít v reálné komunikaci s externími službami. Dále rozšiřuje schopnosti při práci s daty, které byly získány v kapitolách o práci se soubory a databázemi.

Využití HTTP požadavků je klíčové pro tvorbu webových aplikací, automatizaci úkolů a propojení různých systémů – což znamená, že tato kapitola je zásadní pro rozvoj praktických dovedností vývojáře.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


### Příklad 1

Příklad 1: Základní GET request s knihovnou requests

**Název příkladu:** Základní GET požadavek na veřejné API

**Co příklad demonstruje:** Instalace a základní použití knihovny `requests`, provedení GET požadavku na veřejné API.

**Kompletní spustitelný kód:**



**Očekávaný výstup:**
```
Požadavek byl úspěšný!
Status kód: 200
Název článku: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
ID článku: 1
```

**Vysvětlení jak kód funguje:**
- Kód používá knihovnu `requests` pro odeslání GET požadavku na veřejné API (jsonplaceholder.typicode.com)
- Získává odpověď a kontroluje její status kód
- Pokud je status kód 200, znamená to úspěšný požadavek
- Metoda `response.json()` převede JSON odpověď na Python slovník
- Chyby při komunikaci jsou zachyceny pomocí try-except bloku

---

##

In [ ]:
# Instalace knihovny (pouze jednorázově):
# pip install requests

import requests

# Základní GET požadavek na veřejné API
def basic_get_request():
    # URL endpoint pro testování GET požadavku
    url = "https://jsonplaceholder.typicode.com/posts/1"
    
    try:
        # Odeslání GET požadavku
        response = requests.get(url)
        
        # Kontrola, zda byl požadavek úspěšný (status kód 200)
        if response.status_code == 200:
            print("Požadavek byl úspěšný!")
            print(f"Status kód: {response.status_code}")
            
            # Získání dat v JSON formátu
            data = response.json()
            print(f"Název článku: {data['title']}")
            print(f"ID článku: {data['id']}")
        else:
            print(f"Chyba při požadavku: {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"Nastala chyba při komunikaci s API: {e}")

# Spuštění funkce
basic_get_request()

### Příklad 2

Příklad 2: POST request s autentizací a hlavičkami

**Název příkladu:** Odeslání dat na server pomocí POST požadavku

**Co příklad demonstruje:** Vytvoření nového záznamu pomocí POST, práce s hlavičkami a autentizací.

**Kompletní spustitelný kód:**



**Očekávaný výstup:**
```
Článek byl úspěšně vytvořen!
Status kód: 201
Název nového článku: Moje nový článek
ID nového článku: 101
```

**Vysvětlení jak kód funguje:**
- Kód používá POST požadavek pro vytvoření nového záznamu na serveru
- Vytváří slovník s daty, které chceme odeslat
- Nastavuje hlavičky (`Content-Type`, `Accept`) a převádí data do JSON formátu pomocí `json.dumps()`
- Kontroluje status kód 201, což znamená "věc byla úspěšně vytvořena"
- Získává odpověď a zobrazuje vytvořené data

---

##

In [ ]:
import requests
import json

def post_request_with_auth():
    # URL endpoint pro vytvoření nového záznamu
    url = "https://jsonplaceholder.typicode.com/posts"
    
    # Data, která chceme odeslat (v tomto případě pouze testovací data)
    new_post = {
        "title": "Moje nový článek",
        "body": "Toto je tělo mého nového článku vytvořeného pomocí API.",
        "userId": 1
    }
    
    # Nastavení hlaviček (včetně typu obsahu a autentizace)
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
        # V reálném případě by tu mohly být i autentizační hlavičky jako:
        # "Authorization": "Bearer token_hodnota"
    }
    
    try:
        # Odeslání POST požadavku
        response = requests.post(url, data=json.dumps(new_post), headers=headers)
        
        # Kontrola status kódu
        if response.status_code == 201:  # 201 znamená "Created"
            print("Článek byl úspěšně vytvořen!")
            print(f"Status kód: {response.status_code}")
            
            # Získání odpovědi
            created_post = response.json()
            print(f"Název nového článku: {created_post['title']}")
            print(f"ID nového článku: {created_post['id']}")
        else:
            print(f"Chyba při vytváření článku: {response.status_code}")
            print(f"Odpověď: {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"Nastala chyba při komunikaci s API: {e}")

# Spuštění funkce
post_request_with_auth()

### Příklad 3

Příklad 3: Správné zacházení s chybami a status kódy

**Název příkladu:** Správa chyb a status kódů při práci s API

**Co příklad demonstruje:** Práce s různými status kódy, ošetření chyb a zpracování neúspěšných požadavků.

**Kompletní spustitelný kód:**



**Očekávaný výstup:**
```
Chyba 404: Stránka nebyla nalezena
```

**Vysvětlení jak kód funguje:**
- Kód používá speciální testovací API (`httpbin.org`) které odpovídá konkrétním status kódy
- Zpracovává různé status kódy včetně chyb 404, 500 a 429 (rate limiting)
- Používá různé typy `except` bloků pro zachycení různých druhů chyb
- Zobrazuje příslušné zprávy podle status kódu

---

##

In [ ]:
import requests

def handle_errors_and_status_codes():
    # Testovací URL (toto API může odpovídat jinak)
    url = "https://httpbin.org/status/404"
    
    try:
        # Odeslání GET požadavku
        response = requests.get(url)
        
        # Kontrola status kódu a reakce na různé případy
        if response.status_code == 200:
            print("Požadavek byl úspěšný!")
            data = response.json()
            print(f"Data: {data}")
            
        elif response.status_code == 404:
            print("Chyba 404: Stránka nebyla nalezena")
            
        elif response.status_code == 500:
            print("Chyba 500: Interní chyba serveru")
            
        elif response.status_code == 429:
            print("Chyba 429: Příliš mnoho požadavků (Rate limiting)")
            
        else:
            print(f"Neznámý status kód: {response.status_code}")
            print(f"Detaily odpovědi: {response.text}")
            
    except requests.exceptions.ConnectionError:
        print("Chyba připojení k serveru")
        
    except requests.exceptions.Timeout:
        print("Požadavek vypršel (Timeout)")
        
    except requests.exceptions.RequestException as e:
        print(f"Nastala obecná chyba: {e}")

# Spuštění funkce
handle_errors_and_status_codes()

### Příklad 4

Příklad 4: PUT a PATCH požadavky s validací dat

**Název příkladu:** Aktualizace dat pomocí PUT a PATCH požadavku

**Co příklad demonstruje:** Aktualizace existujících záznamů pomocí PUT a PATCH metod.

**Kompletní spustitelný kód:**

```python
import requests
import json

def update_data_with_put_patch():
    # URL endpoint pro aktualizaci záznamu (použijeme testovací API)
    url = "https://jsonplaceholder.typicode.com/posts/1"
    
    # Data pro aktualizaci
    updated_data = {
        "id": 1,
        "title": "Aktualizovaný název článku",
        "body": "Toto je aktualizovaný text článku.",
        "userId": 1
    }
    
    try:
        # PUT požadavek (úplná aktualizace)
        print("Odesílání PUT požadavku...")
        put_response = requests.put(url, data=json.dumps(updated_data), 
                                  headers={"Content-Type": "application/json"})
        
        if put_response.status_code == 200:
            print("PUT požadavek byl úspěšný!")
            updated_post = put_response.json()
            print(f"Aktualizovaný název: {updated_post['title']}")
        else:
            print(f"Chyba při PUT požadavku: {put_response.status_code}")

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# Cvičení: Práce s API a HTTP requests

---

## 1. **Název úkolu**
API klient pro získání informací o uživateli z JSONPlaceholder

### **Detailní zadání**
Vytvoř program, který provede GET požadavek na veřejné API (JSONPlaceholder) a vypíše informace o uživateli s ID 1.

### **Vstupní data/požadavky**
- URL: `https://jsonplaceholder.typicode.com/users/1`
- Metoda: GET
- Nezpracovávej chyby, pouze získání dat

### **Očekávaný výstup**
Výpis uživatelských informací (jméno, email, město, firma) do konzole ve formátu:
```
Jméno: Leanne Graham
Email: Sincere@april.biz
Město: Gwenborough
Firma: Romaguera-Crona
```

### **Hints/Nápověda**
1. Použij knihovnu `requests` pro HTTP požadavky.
2. Získaná data jsou ve formátu JSON, použij metodu `.json()`.
3. Vyber potřebné položky z slovníku (např. `user['name']`, `user['email']`).
4. Vypiš výsledky do konzole pomocí `print()`.

### **Kostra řešení**
```python
import requests

url = "https://jsonplaceholder.typicode.com/users/1"
# TODO: proved GET požadavek a získej data

# TODO: vypiš potřebné informace
```

### **Bonusové rozšíření**
Přidej možnost zadání ID uživatele jako parametr příkazové řádky.

---

## 2. **Název úkolu**
Vytvoř POST požadavek pro vytvoření nového příspěvku

### **Detailní zadání**
Vytvoř program, který provede POST požadavek na API (JSONPlaceholder), aby vytvořil nový příspěvek. Výstupem bude ID nově vytvořeného příspěvku.

### **Vstupní data/požadavky**
- URL: `https://jsonplaceholder.typicode.com/posts`
- Metoda: POST
- Tělo požadavku:
  ```json
  {
    "title": "Test title",
    "body": "This is a test body",
    "userId": 1
  }
  ```

### **Očekávaný výstup**
```
Nový příspěvek vytvořen s ID: 101
```

### **Hints/Nápověda**
1. Použij `requests.post()` s parametrem `json` pro odeslání dat.
2. Zkontroluj, že tělo požadavku je správně formátované JSON.
3. Vypiš ID z odpovědi pomocí `response.json()['id']`.
4. Ujisti se, že URL je správná (včetně `/posts`).

### **Kostra řešení**
```python
import requests

url = "https://jsonplaceholder.typicode.com/posts"
data = {
    "title": "Test title",
    "body": "This is a test body",
    "userId": 1
}
# TODO: proved POST požadavek a vypiš ID nového příspěvku
```

### **Bonusové rozšíření**
Přidej validaci vstupních dat před odesláním.

---

## 3. **Název úkolu**
Zpracuj chyby HTTP požadavků

### **Detailní zadání**
Vytvoř program, který provede GET požadavek na neexistující endpoint a zachytí chybu (např. 404). Program by měl vypsat příslušnou chybovou zprávu.

### **Vstupní data/požadavky**
- URL: `https://jsonplaceholder.typicode.com/users/999` (neexistující uživatel)
- Metoda: GET

### **Očekávaný výstup**
```
Chyba při požadavku: 404 Not Found
```

### **Hints/Nápověda**
1. Použij `try/except` blok pro zachycení chyb.
2. Vypiš status kód pomocí `response.status_code`.
3. Přidej `response.raise_for_status()` pro automatické vyvolání výjimky.
4. Můžeš použít `requests.exceptions.HTTPError`.

### **Kostra řešení**
```python
import requests

url = "https://jsonplaceholder.typicode.com/users/999"
# TODO: proveď požadavek a zachyť chybu
```

### **Bonusové rozšíření**
Přidej logování chyb do souboru.

---

## 4. **Název úkolu**
Odeslání dat přes PUT a ověření změny

### **Detailní zadání**
Vytvoř program, který:
1. Získá existující příspěvek (např. ID 1)
2. Odešle PUT požadavek pro úpravu titulku
3. Znovu získá příspěvek a ověří, že se titulek změnil

### **Vstupní data/požadavky**
- GET URL: `https://jsonplaceholder.typicode.com/posts/1`
- PUT URL: `https://jsonplaceholder.typicode.com/posts/1`
- Nový titulek: `"Upravený titulek"`

### **Očekávaný výstup**
```
Původní titulek: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Nový titulek: Upravený titulek
```

### **Hints/Nápověda**
1. Nejprve proveď GET požadavek a ulož původní titulek.
2. Poté odesli PUT požadavek se změněným titulkem.
3. Znovu získej příspěvek a porovnej titulky.
4. Můžeš použít `requests.put()` s parametrem `json`.

### **Kostra řešení**
```python
import requests

url = "https://jsonplaceholder.typicode.com/posts/1"
# TODO: získej původní titulek, uprav ho a ověř změnu
```

### **Bonusové rozšíření**
Použij `requests.Session()` pro efektivnější práci s více požadavky.

---

## 5. **Název úkolu**
Více požadavků paralelně pomocí ThreadPoolExecutor

### **Detailní zadání**
Vytvoř program, který provede 3 paralelní GET požadavky na různé uživatele (např. ID 1, 2, 3) a vypíše jejich jména.

### **Vstupní data/požadavky**
- URL: `https://jsonplaceholder.typicode.com/users/{id}`
- ID uživatelů: 1, 2, 3

### **Očekávaný výstup**
```
Uživatelé:
Leanne Graham
Ervin Howell
Clementine Bauch
```

### **Hints/Nápověda**
1. Použij `concurrent.futures.ThreadPoolExecutor`.
2. Definuj funkci, která provede GET požadavek.
3. Předej ID uživatelů do vláken.
4. Vypiš jména v pořadí odpovědí.

### **Kostra řešení**
```python
import requests
from concurrent.futures import ThreadPoolExecutor

user_ids = [1, 2, 3]

def fetch_user(user_id):
    # TODO: implementuj funkci pro GET požadavek

# TODO: spus


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# Zdroje pro kapitolu: Práce s API a HTTP requests

## 1. Články a tutoriály

1. **[HTTP Request Methods Explained](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods)** – Oficiální dokumentace MDN vysvětlující základní HTTP metody (GET, POST, PUT, DELETE apod.) a jejich použití v API.
2. **[Understanding HTTP Requests and Responses](https://www.smashingmagazine.com/2018/01/understanding-http-requests-responses/)** – Článek od Smashing Magazine, který se zaměřuje na základy komunikace mezi klientem a serverem pomocí HTTP.
3. **[REST API Tutorial for Beginners](https://www.restapitutorial.com/)** – Rozsáhlý tutoriál o REST API, který vysvětluje koncepty, strukturu a použití.
4. **[Making HTTP Requests in Python with requests Library](https://realpython.com/python-requests/)** – Praktický článek od Real Python, který ukazuje, jak provádět HTTP požadavky v Pythonu pomocí knihovny `requests`.
5. **[HTTP Status Codes Explained](https://httpstatuses.com/)** – Stránka s jednoduchým vysvětlením všech HTTP stavových kódů a jejich významu.

## 2. YouTube videa

1. **[HTTP Requests and APIs - Full Course for Beginners](https://www.youtube.com/watch?v=4J0O8V5ZQpE)** – Délka: 1 hodina 30 minut. Komplexní úvod do HTTP požadavků a práce s API, vhodné pro začátečníky.
2. **[Python Requests Library - Complete Tutorial](https://www.youtube.com/watch?v=J59qQ76Oc4o)** – Délka: 22 minuty. Tutoriál zaměřený na práci s knihovnou `requests` v Pythonu, včetně autentizace a error handlingu.
3. **[REST API with Node.js and Express - Beginner's Tutorial](https://www.youtube.com/watch?v=Oe421EPjeBE)** – Délka: 45 minut. Výukové video o tvorbě REST API v Node.js, vhodné pro základní orientaci v backendu.

## 3. Knihy a oficiální dokumentace

1. **[HTTP: The Definitive Guide](https://www.oreilly.com/library/view/http-the-definitive/9780596803320/)** – Knihu od Davida Gourley a Brianu Totty, která podrobně vysvětluje fungování HTTP protokolu.
2. **[The Ultimate Guide to REST APIs](https://swagger.io/resources/books/the-ultimate-guide-to-rest-apis/)** – Průvodce REST API včetně praktických doporučení a nejlepších postupů.
3. **[Official Requests Documentation (Python)](https://requests.readthedocs.io/en/latest/)** – Oficiální dokumentace knihovny `requests` pro Python, která je velmi podrobná a obsahuje i příklady.

## 4. Praktické projekty

1. **Vytvořit CLI aplikaci pro práci s veřejným API (např. JSONPlaceholder nebo OpenWeatherMap)** – Projekt v Pythonu nebo Node.js, který bude provádět různé HTTP požadavky a zpracovávat odpovědi.
2. **Napsat malou aplikaci s REST API backendem (např. TODO app) pomocí Express.js nebo Flask** – Praktická ukázka tvorby vlastního API se základními CRUD operacemi.

## 5. Online nástroje a playgroundy

1. **[Postman](https://www.postman.com/)** – Výkonný nástroj pro testování API, který umožňuje odesílat HTTP požadav


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Requests library - installation</li>
<li>✓ HTTP methods: GET, POST, PUT, DELETE, PATCH</li>
<li>✓ Headers a authentication</li>
<li>✓ Query parameters a request body</li>
<li>✓ JSON API consumption</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 14 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:16:02*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

